We will be implementing a CNN from scratch heere using just Numpy. Here, I will keep the entire evolution of code until we get a final, polished CNN architecture. This way, we can understand the problems and implementations in a chronological way. 

In [2]:
import numpy as np
import random
import itertools



In [3]:
#Define the fundamentals of CNN:

#Take W size Input Image.
#Use a filter of size K.
#Return output of size W-k+1 (No Padding, No Stride).
#We will use and understand Padding & Stride as we progress into bottlenecks and more complex problems. 
#The main task here is to develop algorithm to convolute the filter through the image.

class con2d:
    def __init__(self, w, k):
        self.W = w
        self.K = k
        self.filter = np.random.rand(k,k)
        self.bias = random.uniform(-1,1)

    def forward(self, X):
        sum = 0
        Y = np.zeros((self.W + 1 - self.K, self.W + 1 - self.K))
        for i in range(self.W + 1 - self.K):
            for j in range(self.W + 1 - self.K):
                patch = X[i:i+self.K, j:j+self.K]
                for a in range(self.K):
                    for b in range(self.K):
                        sum += patch[a,b] * self.filter[a,b]
                #sum += self.bias
                #sum = np.tanh(sum)
                Y[i,j] = sum
                sum = 0
        return Y

In [4]:
input_image = np.array([
    [1, 1, 1, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 1, 1, 1],
    [0, 0, 1, 1, 0],
    [0, 1, 1, 0, 0]
])

kernel = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])

myCNN = con2d(5,3)
myCNN.filter = kernel
myCNN.forward(input_image)

array([[-2.,  0.,  2.],
       [-3., -2.,  2.],
       [-3., -1.,  2.]])

This was an extremely inefficient implementation but we did get correct outcome. For now lets focus on the fact that CNNs work on rgb data which has 3 channels. But our code assumes an image as a flat 2D surface. We have to add depth/channel to our input matrix and thus our kernel. But importantly, our output matrix/activation map must remain the same.

In [5]:
class con2d:
    def __init__(self, w, k, c):
        self.W = w
        self.K = k
        self.C = c #Channel
        self.filter = np.random.rand(k,k,c)
        self.bias = random.uniform(-1,1)

    def forward(self, X):
        sum = 0
        Y = np.zeros((self.W + 1 - self.K, self.W + 1 - self.K))
        for i in range(self.W + 1 - self.K):
            for j in range(self.W + 1 - self.K):
                patch = X[i:i+self.K, j:j+self.K, :]
                for a in range(self.K):
                    for b in range(self.K):
                        for c in range(self.C):
                            sum += patch[a,b,c] * self.filter[a,b,c]
                #sum += self.bias
                #sum = np.tanh(sum)
                Y[i,j] = sum
                sum = 0
        return Y

In [6]:
input_vol = np.zeros((4, 4, 2))
input_vol[:, :, 0] = 1
input_vol[:, :, 1] = 2

kernel = np.zeros((3, 3, 2))
kernel[:, :, 0] = 1
kernel[:, :, 1] = -1

myCNN = con2d(4,3,2)
myCNN.filter = kernel
myCNN.forward(input_vol)

array([[-9., -9.],
       [-9., -9.]])

In [7]:
#To make this complete, lets uncomment bias application and let's try to have more than one filters.
class con2d:
    def __init__(self, w, k, c, n):
        self.W = w
        self.K = k
        self.C = c #Channels
        self.Cn = n #Number of filters
        self.filters = [np.random.rand(k,k,c) for _ in range(n)]
        self.bias = [random.uniform(-1,1) for _ in range(n)]

    def forward(self, X):
        sums = np.zeros(self.Cn)
        Y = [np.zeros((self.W + 1 - self.K, self.W + 1 - self.K)) for _ in range(self.Cn)]
        for i in range(self.W + 1 - self.K):
            for j in range(self.W + 1 - self.K):
                patch = X[i:i+self.K, j:j+self.K, :]
                for a in range(self.K):
                    for b in range(self.K):
                        for c in range(self.C):
                            for index, filter in enumerate(self.filters):
                                sums[index] += patch[a,b,c] * filter[a,b,c]
                for idx, sum in enumerate(sums):
                    sum += self.bias[idx]
                    #sum = np.tanh(sum)
                    Y[idx][i,j] = sum
                    sum = 0
        return Y

But these nested loops are extremely inefficient. We have to adress this. Modern libraries like pytorch and tensorflow use a standard practice called "im2col" where the entire image is stretched into a single vector arranged by the filter size. One way is to then also dilate our filter adding zeros in between such that a single matrix multiplication will simulate the filter being slid over the image. 

This however, would be inefficient. Why? We aren't using multiple nested loops which significantly improve performance. But, we are doing unecessarily large matrix multiplication involving 0 elements.


Best and SoTA practice is to flatten the image locally. I.e. we flatten only the receptive field of the kernel eg. 3x3x3, and then matmul it with our flattened kernel(3x3x3). This results in matmul between a 1x27 image and 27x1 kernel, resulting in a single scalar value. This scalar corresponds to a single pixel of the activation map after being added to the scalar bias term.

Let's take an example. Start from inputs of dimension 16x3x32x32 where 16 is the batch size, 3 is the number of channels and 32x32 is the image size. So we have 16 rgb images of 32x32 resolution as input each batch. Consider a kernel size of 128x3x2x2 where 128 is the number of kernels, 3 represents rgb and 3x3 is the kernel size. 

How does im2col work in this case? 

Firstly, calculate the receptive field size or the output field size.

O = ((I - K + 2P)/S) + 1

This is the standard formula where:
- I : Input image size (eg. 32x32)
- K : Kernel size (eg. 3x3)
- P : Zero Padding size (eg. 0 or 1 if we want O to be 32 = I)
- S : Stride (eg. 1)
- O : Output Activation size

Here, 

O = ((32-3+0)/1) + 1 = 30

So we get activation map of 30x30 per kernel.

If we want the activation map to be same size of that of input, we have to introduce padding: Specifically same padding. 

In our example, same padding requires padding = 1

Therefore we get, O = ((32-3+2)/1) + 1 = 32

For now, let us assume the initial case with no Padding. 

## Im2col Transformation Step-by-Step

### Given:
- Input: (16, 3, 32, 32) - 16 batches, 3 channels, 32×32 images
- Kernels: (128, 3, 3, 3) - 128 output channels/kernels/activation maps, 3 input channels, 3×3 spatial
- Output size: O = ((32 - 3 + 0) / 1) + 1 = **30×30**

### Step 1: Extract All Receptive Fields

For each image in the batch:
- Each receptive field: 3×3×3 = **27 elements**
- Number of positions: 30×30 = **900 positions**
- Total number of flattened matrices that represent all positions: **(900, 27)** (But this flatenning requires special handling so that we differentiate from 27 contigious memory of the image, and 27 neighbouring pixels that constitute a receptive field.)

For the entire batch (16 images):
- We stack all of them vertically (16 × 900, 27) = **(14,400, 27)**

### Step 2: Flatten All 128 Kernels

- Each kernel: 3×3×3 = 27 elements
- Kernels shape: **(27, 128)**

Why? 27 weights for each 128 kernels. 

### Step 3: Giant MatMul

(14,400, 27) @ (27, 128) = **(14,400, 128)**

### Step 4: Reshape and Add Bias

Reshape (14,400, 128) → (16, 128, 30, 30)
Add bias (128,) → broadcasts to (16, 128, 30, 30)


## So lets draw some conclusions first!

1. We need special type of matrix flatenning such that all pixels of local receptive fields(3x3x3 = 27) are in one dimension(x, 27) and all such local receptive fields(30x30 = 900) are in another, batch dimension(900, x) which represents the number of such possible receptive fields. This corresponds to number of pixels in activation map. Finally we need to consider all the receptive fields of all images in the batch. This doesnot create new dimension (not, eg. 16x900x27), but we stack all the receptive fields of all the images vertically. (i.e 900x16 = 14,400). Therefore we get final flatenned 2D matrix of shape = 14400, 27.

    ## Therefore:
    Local receptive field pixels must be arranged non contigiously relative to the image.

    eg.

    11  12  13

    21  22  23

    31  32  33

    should not be flattened as: 

    [11, 12, 13, 14],
        
    [13, 21, 22, 23], 


    [22, 23, 31, 32], 

    [31, 32, 33, 11] 

    Shape(4x4)

    or,

    [1,2,3,4,5,6,7,8,9] : Shape (9)

    But as:
    
    [11, 12, 21, 22],

    [12, 13, 22, 23],

    [21, 22, 23, 31],

    [22, 23, 32, 33]

    Shape(4x4)


2. To define our convolution layer, we need to know some parameters:
- Depth/No. of Channels of Kernel which corresponds to input channels(in_channels)
- Depth/No. of Channels of Activation/Output which corresponds to number of kernels (out_channels)
- Kernel size (kernel_size eg.(2) -> 2x2) 

In [8]:
class tensor:
    def __init__(self, fromArray=np.zeros((2,2)), _children = (), _operation = ''):
        fromArray = fromArray if isinstance(fromArray, np.ndarray) else np.array(fromArray)
        #assert len(fromArray.shape) == 2, "Only 2D Tensors or Scalar to 2D Supported!"
        self.matrix = fromArray
        #self.rows = fromArray.shape[0]
        #self.columns = fromArray.shape[1]
        self.shape = fromArray.shape
        self._prev = set(_children)
        self._operation = _operation
        self._backward = lambda : None
        self.grad = None


    def __repr__(self):
        return f"Tensor Values = {self.matrix}"
    
    @classmethod
    def zeros(cls, shape, dtype = np.float32):
        t = tensor()
        t.matrix = np.zeros(shape, dtype=dtype)
        t.shape = shape
        #t.rows = rows
        #t.columns = columns
        return t
    
    @classmethod
    def random(cls, shape, dtype = np.float32):
        t = tensor()
        t.matrix = (np.random.randn(*shape) * 0.1).astype(dtype=dtype)
        t.shape = shape
        return t
    
    @classmethod
    def const(cls, shape, constant=1, dtype = np.float32):
        t = tensor()
        t.matrix = (np.full(shape, constant)).astype(dtype=dtype)
        t.shape = shape
        #t.rows = rows
        #t.columns = columns
        return t
    
    #Operations
    def __add__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix + other.matrix

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            other.grad = np.zeros_like(other.matrix) if other.grad is None else other.grad
            out1 = self.return_unbroadcasted(out)
            out2 = other.return_unbroadcasted(out)
            self.grad += out1 #Derivation in the notes. 
            other.grad += out2
        out = tensor(out_matrix, (self, other), '+')
        out._backward = _backward
        return out
    
    def __radd__(self, other):
        return self + other
    
    def __sub__(self, other):
        other = self.checkOther(other)
        return self + (-1 * other)
    
    
    def __rsub__(self, other):
        other = self.checkOther(other)
        return other + (-1 * other)
    

    def __mul__(self, other):
        other = self.checkOther(other)
        out_matrix = self.matrix * other.matrix
        def _backward():
            self.grad = np.zeros_like(out.grad) if self.grad is None else self.grad
            other.grad = np.zeros_like(out.grad) if other.grad is None else other.grad
            out1 = self.return_unbroadcasted(out)
            out2 = other.return_unbroadcasted(out)
            self.grad += out1* other.matrix #Derivation in the notes. 
            other.grad += out2 * self.matrix

        out = tensor(out_matrix, (self, other), '*')
        out._backward = _backward
        return out
    
    '''
    batch multiplication might cause shape broadcasts.
    eg. (3,2,2) @ (1,2,3) = (3,2,3)
    this is similar to our element wise operations
    thus we should be handling this the same way we did for elementwise operations
    But, for now, we would be working in a controlled way (Even for CNNS)
    and wouldn't need this handling.
    '''
    def __matmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        assert other.shape[-2] == self.shape[-1], "Dimension Unsupported for @"
        out_matrix = self.matrix @ other.matrix
        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            other.grad = np.zeros_like(other.matrix) if other.grad is None else other.grad
            self.grad += out.grad @ (other.matrix).swapaxes(-2,-1)#Derivation in the notes.
            other.grad += (self.matrix).swapaxes(-2,-1) @ out.grad 
        out = tensor(out_matrix, (self, other), '@')
        out._backward = _backward
        return out
    

    #I and thus we should learn at this point that to make our class compatible for ND tensors,
    #We need the matrix multiplication and Transpose backward to change
    #For higher dimensions, matmul = batch matmul where multiplication is done 
    #along each and every batches of 2D matrix. 
    #eg. If we have (2,3,3) shape tensor, it implies there are two batches of (3,3) matrices
    #similarly, (2,3,3,2) shape = 2x3 batches of 3x2 matrices.
    #matrix multiplication, (2,3,3) @ (2,3,2) = (2,3,2)
    def swap_axes(self, axis1, axis2):
        out_matrix = self.matrix.swapaxes(axis1, axis2)
        
        def _backward():
            self.grad = np.zeros_like(out.grad.swapaxes(axis1,axis2)) if self.grad is None else self.grad
            self.grad += (out.grad).swapaxes(axis1,axis2) #Not in note, but can be derived similarly.

        out = tensor(out_matrix, (self, ), 'T')
        out._backward = _backward

        return out

    def transpose(self):
        out_matrix = self.matrix.transpose()
        
        def _backward():
            self.grad = np.zeros_like(out.grad.transpose()) if self.grad is None else self.grad
            self.grad += (out.grad).transpose() #Not in note, but can be derived similarly.

        out = tensor(out_matrix, (self, ), 'T')
        out._backward = _backward

        return out
    
    def __rmatmul__(self, other):
        other = other if isinstance(other, tensor) else tensor(other)
        return other @ self
    
    def __pow__(self, N):
        assert isinstance(N, int | float), "Can only power up by scalars!"
        out_matrix = self.matrix ** N

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            out1 = self.return_unbroadcasted(out)
            self.grad += N * (self.matrix ** (N-1)) * out1
        
        out = tensor(out_matrix, _children=(self, ), _operation="**")
        out._backward = _backward
        return out
    
    def __truediv__(self, other):
        other = self.checkOther(other)
        return self * (other**-1)
    
    def __rtruediv__(self, other):
        return other * (self**-1)
    
    def sum(self):
        out_matrix = np.array(([[self.matrix.sum()]]))

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += np.ones_like(self.matrix) * out.grad

        out = tensor(out_matrix, _children=(self, ), _operation='sum()')
        out._backward = _backward
        return out

    def mean(self):
        N = np.prod(self.shape)
        out_matrix = np.array(([[self.matrix.sum()/(N)]]))

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += np.ones_like(self.matrix) * out.grad / N

        out = tensor(out_matrix, _children=(self, ), _operation='mean()')
        out._backward = _backward
        return out
    
    def ReLU(self):
        out_matrix = np.maximum(0,self.matrix)

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += (self.matrix > 0).astype(self.matrix.dtype) * out.grad

        out = tensor(out_matrix, (self, ), "ReLU")
        out._backward = _backward
        return out
    
    def reshape(self, shape):
        assert isinstance(shape, tuple), f"Can only reshape using shape tuples e.g. (3,3). Provided is {shape}"
        out_matrix = self.matrix.reshape(shape)

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += out.grad.reshape(self.shape)

        out = tensor(out_matrix, (self, ), "reshape()")
        out._backward = _backward
        return out
    
    def flatten(self):
        out_matrix = self.matrix.reshape(-1,np.prod(self.shape[1:]))

        def _backward():
            self.grad = np.zeros_like(self.matrix) if self.grad is None else self.grad
            self.grad += out.grad.reshape(self.shape)

        out = tensor(out_matrix, (self, ), "flatten()")
        out._backward = _backward
        return out
    
    #Helper Functions
    #def shape(self):
     #   return (self.rows, self.columns)

    def return_unbroadcasted(self, out):  
        added_axis = []
        stretched_axis = []
        for index, (first_no, second_no) in enumerate(itertools.zip_longest(reversed(self.shape), reversed(out.shape))):
            if first_no is None:
                added_axis.append(index)
            elif (first_no == 1) and (second_no > 1):
                stretched_axis.append(index)
        grad = out.grad
        ndim = len(out.shape)
        if stretched_axis:
            original_axes = tuple(ndim - 1 - i for i in stretched_axis)
            grad = np.sum(grad, axis=original_axes, keepdims=True)
        if added_axis:
            original_axes = tuple(ndim - 1 - i for i in added_axis)
            grad = np.sum(grad, axis=original_axes, keepdims=False)
        return grad

    def checkOther(self, other):
        if isinstance(other, int | float):
            other = tensor.const(self.shape, other)
        elif not isinstance(other, tensor):
            other = tensor(other)
        #assert other.shape == self.shape, "Operand Tensor sizes dont match"

        return other
    
    def zero_grad(self):
        self.grad = None
        
    def backward(self):
        self.grad = np.ones_like(self.matrix, dtype=float)
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        for current in reversed(topo):

            current._backward()

    __array_ufunc__ = None

In [9]:
class Conv2d:
    def __init__(self, in_channels, out_channels, kernel_size):
        self.kernel = tensor.random((out_channels, in_channels, kernel_size, kernel_size))

    @classmethod
    def im2col(cls, X : tensor, kernel_size, stride):

        batch_size = X.shape[0]
        channels = X.shape[1]
        image_height = X.shape[-2] #Rows
        image_width = X.shape[-1] #Columns


        #We are assuming square kernels.
        kernel_h = kernel_size
        kernel_w = kernel_size

        act_h = (((image_height - kernel_size)//stride) + 1) #height of activation
        act_w = (((image_width - kernel_size)//stride) + 1)  #width of activation

        istrides = X.matrix.strides #strides of input tensor

        intermediate_6D = np.lib.stride_tricks.as_strided(
                            X.matrix,
                            shape=(batch_size, act_h, act_w, channels, kernel_h, kernel_w),
                            strides=(istrides[0], #No of images stride bytes
                                     istrides[-2] * stride, #Activation map Vertical stride bytes
                                     istrides[-1] * stride, #Activation map Horizontal stride bytes
                                     istrides[1], #Channel stride bytes
                                     istrides[-2], #Rective field vertical stride bytes
                                     istrides[-1]) #Receptive field horizontal stride bytes
                            )
        
        out_shape = (batch_size * act_h * act_w, channels * kernel_h * kernel_w)
        out_matrix = np.reshape(intermediate_6D, shape=out_shape)


        def _backward():
            X.grad = np.zeros_like(X.matrix) if X.grad is None else X.grad
            grad_6D = np.reshape(out.grad, shape=(batch_size, act_h, act_w, channels, kernel_h, kernel_w,))

            #For each element in this 6D tensor, having 6D index, we have to calculate the coresponding 4D index.
            #The formula has been conceptually derived in the notes.
            #Here, we first generate all the indices of the 6D tensor and store each index dimension in separate list
            #Then using the derived formula, we batch convert the 6D indices to 4D indices.

            batch = np.arange(batch_size).reshape(batch_size,1,1,1,1,1)
            field_h = np.arange(act_h).reshape(1,act_h,1,1,1,1)
            field_w = np.arange(act_w).reshape(1,1,act_w,1,1,1)
            channel = np.arange(channels).reshape(1,1,1,channels,1,1)
            k_h = np.arange(kernel_h).reshape(1,1,1,1,kernel_h,1)
            k_w = np.arange(kernel_w).reshape(1,1,1,1,1,kernel_w)

            x = stride * field_h + k_h
            y = stride * field_w + k_w

            np.add.at(X.grad, (batch, channel, x, y), grad_6D)

        out = tensor(out_matrix, _children=(X, ), _operation='im2col')
        out._backward = _backward

        return out
    
    def maxpool2D()
        

SyntaxError: expected ':' (443750374.py, line 64)

In [ ]:
batch = tensor(np.random.randn(16, 3, 5, 5).astype(float))

print("Input Shape: ", batch.shape)

im2col = Conv2d.im2col(batch, 3, 2)

print("Output Shape: ", im2col.shape)

Input Shape:  (16, 3, 5, 5)
Output Shape:  (64, 27)


This was a valid approach. Elegant and shows the direct use of formula. But, we have an even more efficient approach. It involves slicing. In this method, instead of creating large index array which requires both time and memory, we can call 2 simple for loops to loop over the kernel sizes and slice the out.grad array for each position of the kernel index.

In [ ]:
class Conv2d:
    def __init__(self, in_channels=1, out_channels=1, kernel_size=2):
        self.kernel = tensor.random((out_channels, in_channels, kernel_size, kernel_size))
        self.bias = tensor.random((out_channels, ))
        self.kernel_size = kernel_size
        self.out_channels = out_channels
        self.in_channels = in_channels

    def __call__(self, X : tensor, stride=1):

        batch_size = X.shape[0]

        X_col, act_h, act_w = Conv2d.im2col(X, kernel_size=self.kernel_size, stride=stride)
        K_col_shape = (self.out_channels, self.kernel_size*self.kernel_size*self.in_channels)
        K_col = self.kernel.reshape(K_col_shape).transpose()
        Y_col = X_col @ K_col + self.bias
        Y = Y_col.reshape((batch_size, self.out_channels, act_h, act_w))

        return Y
        
    @classmethod
    def im2col(cls, X : tensor, kernel_size=2, stride=1):

        batch_size = X.shape[0]
        channels = X.shape[1]
        image_height = X.shape[-2] #Rows
        image_width = X.shape[-1] #Columns


        #We are assuming square kernels.
        kernel_h = kernel_size
        kernel_w = kernel_size

        act_h = (((image_height - kernel_size)//stride) + 1) #height of activation
        act_w = (((image_width - kernel_size)//stride) + 1)  #width of activation

        istrides = X.matrix.strides #strides of input tensor

        intermediate_6D = np.lib.stride_tricks.as_strided(
                            X.matrix,
                            shape=(batch_size, act_h, act_w, channels, kernel_h, kernel_w),
                            strides=(istrides[0], #No of images stride bytes
                                     istrides[-2] * stride, #Activation map Vertical stride bytes
                                     istrides[-1] * stride, #Activation map Horizontal stride bytes
                                     istrides[1], #Channel stride bytes
                                     istrides[-2], #Rective field vertical stride bytes
                                     istrides[-1]) #Receptive field horizontal stride bytes
                            )
        
        out_shape = (batch_size * act_h * act_w, channels * kernel_h * kernel_w)
        out_matrix = np.reshape(intermediate_6D, shape=out_shape)


        def _backward():
            X.grad = np.zeros_like(X.matrix) if X.grad is None else X.grad
            
            grad_6D = out.grad.reshape(batch_size, act_h, act_w, channels, kernel_h, kernel_w)

            for i in range(kernel_h):
                for j in range(kernel_w):
                    # 1. Extract the gradient slice for this kernel position
                    grad_slice = grad_6D[:, :, :, :, i, j]
                    
                    grad_slice_transposed = grad_slice.transpose(0, 3, 1, 2)
                    
                    X.grad[:, :, 
                        i : i + act_h * stride : stride, 
                        j : j + act_w * stride : stride
                    ] += grad_slice_transposed

        out = tensor(out_matrix, _children=(X, ), _operation='im2col')

        out._backward = _backward

        return out, act_h, act_w
    
    def maxpool2d(cls, Y: tensor, pool_size, stride):   
        batch_size = Y.shape[0]
        channels = Y.shape[1]
        image_height = Y.shape[-2] #Rows
        image_width = Y.shape[-1] #Columns


        #We are assuming square kernels.
        pool_h = pool_size
        pool_w = pool_size

        act_h = (((image_height - pool_h)//stride) + 1) #height of activation
        act_w = (((image_width - pool_w)//stride) + 1)  #width of activation

        istrides = Y.matrix.strides #strides of input tensor

        intermediate_6D = np.lib.stride_tricks.as_strided(
                            Y.matrix,
                            shape=(batch_size, act_h, act_w, channels, pool_h, pool_w),
                            strides=(istrides[0], #No of images stride bytes
                                     istrides[-2] * stride, #Activation map Vertical stride bytes
                                     istrides[-1] * stride, #Activation map Horizontal stride bytes
                                     istrides[1], #Channel stride bytes
                                     istrides[-2], #Rective field vertical stride bytes
                                     istrides[-1]) #Receptive field horizontal stride bytes
                            )
        



NameError: name 'tensor' is not defined

In [34]:
# Test im2col gradient via Conv2d layer
X = tensor(np.random.randn(1, 1, 4, 4))
layer = Conv2d(in_channels=1, out_channels=1, kernel_size=2)
Y = layer(X, stride=1)
Y.backward()
print("Input gradient:\n", X.grad[0, 0])

Input gradient:
 [[-0.1098557   0.00380984  0.00380984  0.11366554]
 [ 0.00869901  0.11100636  0.11100636  0.10230735]
 [ 0.00869901  0.11100636  0.11100636  0.10230735]
 [ 0.1185547   0.10719652  0.10719652 -0.01135818]]


In [ ]:
batch_number = 1
filters = 1
act_h = 4
act_w = 4

elements_in_activation_map = batch_number*filters*act_h*act_w #(16)

numbers = np.random.randint(0,elements_in_activation_map, elements_in_activation_map) # (0,16)
Y = numbers.reshape(batch_number*act_h*act_w, filters) #(16,1)
print(Y)
print("Strides of Y=", Y.strides)
print("-" * 10)


istrides = Y.strides

pool_h = 2
pool_w = 2
pool_total = pool_h * pool_w #4
stride = 1

pooled_h = (act_h - pool_h)//stride + 1 #3
pooled_w = (act_w - pool_w)//stride + 1 #3

intermediate_6D = np.lib.stride_tricks.as_strided(
                    Y,
                    shape = (batch_number, filters, pooled_h, pooled_w, pool_h, pool_w),
                    strides = (act_h*act_w*istrides[0], #4x4x8 = 128 #16 elements down
                               istrides[1], #8 #1 element right
                               act_w*istrides[0], #4*8 = 32 #4 elements down
                               istrides[0], #8 #1 element down
                               act_w*istrides[0], #4*8 = 32 #4 elements down
                               istrides[0]
                    )
)

print(intermediate_6D)
print("-" * 10)

intermediate_5D = np.reshape(intermediate_6D, (batch_number, filters, pooled_h, pooled_w, pool_total))


print(intermediate_5D)
print("-" * 10)
Pooled_output = np.max(intermediate_5D, axis = -1)
print(Pooled_output)
print("-" * 10)
IndexA_for5D = np.argmax(intermediate_5D, axis = -1)
print(IndexA_for5D)

def _backward():
    # Recover window position (i, j) from flat index in last dim
    flat_idx = IndexA_for5D  # (B, F, pooled_h, pooled_w)
    i = flat_idx // pool_w
    j = flat_idx % pool_w

    # Build grids for pooled positions and filters
    ph_grid = np.arange(pooled_h).reshape(1, 1, pooled_h, 1)
    pw_grid = np.arange(pooled_w).reshape(1, 1, 1, pooled_w)
    f_grid = np.arange(filters).reshape(1, filters, 1, 1)

    # Broadcast to match shape
    ph = np.broadcast_to(ph_grid, flat_idx.shape)
    pw = np.broadcast_to(pw_grid, flat_idx.shape)
    f_idx = np.broadcast_to(f_grid, flat_idx.shape)

    # Row index inside each batch: (ph + i) * act_w + (pw + j)
    row_no_batch = (ph + i) * act_w + (pw + j)

    # Batch offsets in the flattened Y first dimension
    b_offset = (np.arange(batch_number).reshape(batch_number, 1, 1, 1) * (act_h * act_w))
    b_offset = np.broadcast_to(b_offset, flat_idx.shape)

    # Final row indices in Y
    rows = b_offset + row_no_batch  # (B, F, pooled_h, pooled_w)

    # Accumulate gradients back into Y using vectorized scatter
    np.add.at(Y_grad, (rows.ravel(), f_idx.ravel()), grad_out.ravel())():



[[ 2]
 [ 5]
 [ 7]
 [10]
 [10]
 [ 1]
 [ 4]
 [13]
 [ 0]
 [11]
 [14]
 [15]
 [ 0]
 [ 4]
 [11]
 [12]]
Strides of Y= (8, 8)
----------
[[[[[[ 2  5]
     [10  1]]

    [[ 5  7]
     [ 1  4]]

    [[ 7 10]
     [ 4 13]]]


   [[[10  1]
     [ 0 11]]

    [[ 1  4]
     [11 14]]

    [[ 4 13]
     [14 15]]]


   [[[ 0 11]
     [ 0  4]]

    [[11 14]
     [ 4 11]]

    [[14 15]
     [11 12]]]]]]
----------
[[[[[ 2  5 10  1]
    [ 5  7  1  4]
    [ 7 10  4 13]]

   [[10  1  0 11]
    [ 1  4 11 14]
    [ 4 13 14 15]]

   [[ 0 11  0  4]
    [11 14  4 11]
    [14 15 11 12]]]]]
----------
[[[[10  7 13]
   [11 14 15]
   [11 14 15]]]]
----------
[[[[2 1 3]
   [3 3 3]
   [1 1 1]]]]


In [62]:
# Route 4D downstream gradient back to 2D upstream using 5D indices
# Downstream gradient from pooled output (B, F, pooled_h, pooled_w)
grad_out = np.ones((batch_number, filters, pooled_h, pooled_w), dtype=float)

# Upstream gradient buffer for Y (B*act_h*act_w, F)
Y_grad = np.zeros_like(Y, dtype=float)

# Recover window position (i, j) from flat index in last dim
flat_idx = IndexA_for5D  # (B, F, pooled_h, pooled_w)
i = flat_idx // pool_w
j = flat_idx % pool_w

# Build grids for pooled positions and filters
ph_grid = np.arange(pooled_h).reshape(1, 1, pooled_h, 1)
pw_grid = np.arange(pooled_w).reshape(1, 1, 1, pooled_w)
f_grid = np.arange(filters).reshape(1, filters, 1, 1)

# Broadcast to match shape
ph = np.broadcast_to(ph_grid, flat_idx.shape)
pw = np.broadcast_to(pw_grid, flat_idx.shape)
f_idx = np.broadcast_to(f_grid, flat_idx.shape)

# Row index inside each batch: (ph + i) * act_w + (pw + j)
row_no_batch = (ph + i) * act_w + (pw + j)

# Batch offsets in the flattened Y first dimension
b_offset = (np.arange(batch_number).reshape(batch_number, 1, 1, 1) * (act_h * act_w))
b_offset = np.broadcast_to(b_offset, flat_idx.shape)

# Final row indices in Y
rows = b_offset + row_no_batch  # (B, F, pooled_h, pooled_w)

# Accumulate gradients back into Y using vectorized scatter
np.add.at(Y_grad, (rows.ravel(), f_idx.ravel()), grad_out.ravel())

print("Upstream gradient (Y_grad):")
#print(Y_grad.reshape(batch_number, act_h, act_w, filters).transpose(0, 3, 1, 2))
print(Y_grad)
print("Note: Only max positions per 2x2 window receive gradient.")

Upstream gradient (Y_grad):
[[0.]
 [0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [2.]
 [2.]
 [2.]
 [0.]
 [0.]
 [0.]
 [0.]]
Note: Only max positions per 2x2 window receive gradient.


In [49]:
numbers = np.random.random_integers(0,16, 16)
Y = numbers.reshape(16,1)
act_h = 4
act_w = 4
istrides = Y.strides
istrides

/var/folders/vx/0mwc7nyn64x7k2f0jlg9hs300000gn/T/ipykernel_26325/2651310694.py:1: DeprecationWarning: This function is deprecated. Please call randint(0, 16 + 1) instead
  numbers = np.random.random_integers(0,16, 16)


(8, 8)